In [1]:
import pandas as pd
import h2o
import utm
from utils.from_latlon import from_latlon

In [2]:
lat = -22.3846
longitude = 156.32847

easting, northing, zone_num, zone_letter = from_latlon(lat, longitude)

print(easting)

224920.91212243412


In [3]:
x = 3
y = 4
print(add(x, y))

7


In [ ]:
pred_data = 'Example_data\Input\Lotek_combined_csv\sampled_data.csv'

# Variable parameters
freq = '5min' # Frequency of data

dimensions = ['xOffset', 'yOffset']
predictors = ['ant1_mean', 'ant2_mean', 'ant3_mean', 'ant4_mean', 'ant1_count', 'ant2_count', 'ant3_count', 'ant4_count', 'ant1_std', 'ant2_std', 'ant3_std', 'ant4_std', 'mean_std', 'total_count']

In [ ]:
# Get prediction data
pred_data = pd.read_csv(pred_data)
pred_data['DateAndTime'] = pd.to_datetime(train_data['DateAndTime'])

In [ ]:
# Preprocess the training and testing data
train_data_preproc = preprocess_sim_data(train_data, data_type, freq, tower_locs)
test_data_preproc = preprocess_sim_data(test_data, data_type, freq, tower_locs)

# Initialise h2o
h2o.init()

# Train, save and test the models for each dimension
for dimension in dimensions:
    print(f"Training model for {dimension}")
    # Train the model
    variables = predictors + [dimension]
    training_input = train_data_preproc[variables]
    train = h2o.H2OFrame(training_input)
    aml = H2OAutoML(max_models=20, seed=seed, stopping_metric='MAE', sort_metric='MAE')
    aml.train(x=predictors, y=dimension, training_frame=train)
    print(aml.leaderboard)

    # Save the leader model
    h2o.save_model(aml.leader, path = f"Example_data\Output\Trained_models\{dimension}", force=True)

    # Make predictions on the test data
    test = h2o.H2OFrame(test_data_preproc)
    preds = aml.leader.predict(test)

    # Save predictions to a new column in the test dataframe
    pred_column_name = f"{dimension}_pred"
    test_data_preproc[pred_column_name] = preds.as_data_frame()

# Stop h2o
h2o.cluster().shutdown()